# Modelling intertidal elevation using tidal data <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[ga_ls5t_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[ga_ls7e_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)
* **Prerequisites:** For more information about the tidal modelling step of this analysis, refer to the [Tidal modelling notebook](../Frequently_used_code/Tidal_modelling.ipynb)

## Background
Intertidal environments support important ecological habitats (e.g. sandy beaches and shores, tidal flats and rocky shores and reefs), and provide many valuable benefits such as storm surge protection, carbon storage and natural resources for recreational and commercial use. 
However, intertidal zones are faced with increasing threats from coastal erosion, land reclamation (e.g. port construction), and sea level rise. 
Accurate elevation data describing the height and shape of the coastline is needed to help predict when and where these threats will have the greatest impact. 
However, this data is expensive and challenging to map across the entire intertidal zone of a continent the size of Australia.

### Digital Earth Australia use case
The rise and fall of the tide can be used to reveal the three-dimensional shape of the coastline by mapping the boundary betweeen water and land across a range of known tides (e.g. from low tide to high tide). 
Assuming that the land-water boundary is a line of constant height relative to mean sea level (MSL), elevations can be modelled for the area of coastline located between the lowest and highest observed tide. 

Imagery from satellites such as the NASA/USGS Landsat program is available for free for the entire planet, making satellite imagery a powerful and cost-effective tool for modelling the 3D shape and structure of the intertidal zone at regional or national scale. 
Recently, Geoscience Australia combined 30 years of Landsat data from the Digital Earth Australia archive with tidal modelling to produce the first 3D model of Australia's entire coastline: the **National Intertidal Digital Elevation Model** or NIDEM (for more information, see [Bishop-Taylor et al. 2019](https://doi.org/10.1016/j.ecss.2019.03.006) or [access the dataset here](http://pid.geoscience.gov.au/dataset/ga/123678)). 

## Description

In this example, we demonstrate a simplified version of the NIDEM method that combines data from the Landsat 5, 7 and 8 satellites with tidal modelling, image compositing and spatial interpolation techniques. 
We first map the boundary between land and water from low to high tide, and use this information to generate smooth, continuous 3D elevation maps of the intertidal zone. 
The resulting data may assist in mapping the habitats of threatened coastal species, identifying areas of coastal erosion, planning for extreme events such as storm surges and flooding, and improving models of how sea level rise will affect the Australian coastline. 
This worked example takes users through the code required to:

1.  Load in a cloud-free Landsat time series
2.  Compute a water index (NDWI)
3.  Tag and sort satellite images by tide height
4.  Create "summary" or composite images that show the distribution of land and water at discrete intervals of the tidal range (e.g. at low tide, high tide)
5.  Extract and visualise the topography of the intertidal zone as depth contours
6.  Interpolate depth contours into a smooth, continuous Digital Elevation Model (DEM) of the intertidal zone

***


## Getting started
**To run this analysis**, run all the cells in the notebook, starting with the "Load packages" cell.

**After finishing the analysis**, return to the "Analysis parameters" cell, modify some values (e.g. choose a different location or time period to analyse) and re-run the analysis.
There are additional instructions on modifying the notebook at the end.

### Load packages
Load key Python packages and supporting functions for the analysis.

In [ ]:
%matplotlib inline

import datacube
import sys
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datacube.utils.cog import write_cog

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.datahandling import load_ard, mostcommon_crs
from dea_tools.plotting import rgb, map_shapefile, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.spatial import subpixel_contours, interpolate_2d, contours_to_arrays
from dea_tools.coastal import tidal_tag
from dea_tools.dask import create_local_dask_cluster

# Create local dask cluster to improve data load time
create_local_dask_cluster()


### Connect to the datacube
Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [ ]:
dc = datacube.Datacube(app='Intertidal_elevation')

### Analysis parameters

The following cell set important parameters for the analysis:

* `lat_range`: The latitude range to analyse (e.g. `(-27.60, -27.665)`). 
For reasonable load times, keep this to a range of ~0.1 degrees or less.
* `lon_range`: The longitude range to analyse (e.g. `(153.33, 153.425)`). 
For reasonable load times, keep this to a range of ~0.1 degrees or less.
* `time_range`: The date range to analyse (e.g. `('2000', '2019') `)

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.
The example generates an intertidal elevation model for intertidal flats in Moreton Bay south of Brisbane, Australia.

To ensure that the tidal modelling part of this analysis works correctly, please make sure the **centre of the study area is located over water** when setting `lat_range` and `lon_range`.


In [ ]:
lat_range = (-17.5, -17.6)
lon_range = (139.70, 139.86)

# lat_range = -16.8511, -16.9165
# lon_range = 138.8710, 139.0400

# lat_range = -15.7582, -15.94627
# lon_range = 136.63593, 136.9977

time_range = ('2016', '2021')



## View the selected location
The next cell will display the selected area on an interactive map.
Feel free to zoom in and out to get a better understanding of the area you'll be analysing.
Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [ ]:
display_map(x=lon_range, y=lat_range)


## Load cloud-masked Landsat data
The first step in this analysis is to load in Landsat data for the `lat_range`, `lon_range` and `time_range` we provided above. 
The code below first connects to the datacube database, and then uses the `load_ard` function to load in data from the Landsat 5, 7 and 8 satellites for the area and time included in `lat_range`, `lon_range` and `time_range`. 
We set `min_gooddata=0.9` to load only satellite images containing less than 10% cloud, allowing us to focus on satellite images that contain useful data. 

In [ ]:
# Create the 'query' dictionary object, which contains the longitudes, 
# latitudes and time provided above
query = {
    'y': lat_range,
    'x': lon_range,
    'time': time_range,
    'measurements': ['nbart_green', 'nbart_nir_1', 'fmask'],
    'resolution': (-10, 10),
}

# # Identify the most common projection system in the input query 
# output_crs = mostcommon_crs(dc=dc, product='ga_ls8c_ard_3', query=query)

# # Load available data from all three Landsat satellites
# landsat_ds = load_ard(dc=dc, 
#                       products=['ga_ls5t_ard_3', 
#                                 'ga_ls7e_ard_3', 
#                                 'ga_ls8c_ard_3'],
#                       output_crs=output_crs,
#                       align=(15, 15),
#                       group_by='solar_day',
#                       min_gooddata=0.9,
#                       dask_chunks={'time': 1, 'x': 2000, 'y': 2000},
#                       **query)

# Identify the most common projection system in the input query 
output_crs = mostcommon_crs(dc=dc, product='s2a_ard_granule', query=query)

landsat_ds = load_ard(dc=dc, 
#                       products=['ga_ls5t_ard_3', 
#                                 'ga_ls7e_ard_3', 
#                                 'ga_ls8c_ard_3'],
                      products=['s2a_ard_granule', 
                                's2b_ard_granule'], 
                      output_crs=output_crs,
                      mask_pixel_quality=False,
#                       align=(15, 15),
                      group_by='solar_day',
                      min_gooddata=0.0,
                      resampling={'*': 'cubic', 'fmask': 'nearest'},
                      dask_chunks={'time': 1, 'x': 2048, 'y': 2048},
                      **query)

landsat_ds.load()


In [ ]:
import odc.algo
from skimage.filters import threshold_local, threshold_otsu

# Extract boolean mask
mask_clouds = odc.algo.enum_to_bool(
    landsat_ds.fmask, categories=['cloud', 'shadow'])

# Nodata
mask_nodata = odc.algo.enum_to_bool(
    landsat_ds.fmask, categories=['nodata'])

# Calculate mean cloudiness
mask_clouds_mean = mask_clouds.mean(dim='time')

# Calculate OTSU threshold and create mask
thresh_otsu = threshold_otsu(mask_clouds_mean.values)
mask_clouds_persistent = (mask_clouds_mean > thresh_otsu)
mask_clouds_persistent.plot()

# # Close mask to remove small holes in cloud, open mask to
# # remove narrow false positive cloud, then dilate
# mask_cleaned = odc.algo.mask_cleanup(mask=mask,
#                                      mask_filters=[('erosion', 5)])




In [ ]:
i = 159
mask_clouds.isel(time=i).plot()

In [ ]:
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage as nd
from skimage.measure import regionprops

In [ ]:
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt

# Sample study area array
false_positives = np.array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
                            [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
                            [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
                            [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
                            [1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
                            [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1],
                            [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                            [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0],
                            [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                            [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

# Plot example array
plt.imshow(false_positives, cmap="jet", interpolation='nearest')

clouds = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1],
                   [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1],
                   [1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0],
                   [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1],
                   [1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1],
                   [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
                   [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0],
                   [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                   [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])

# Plot seeds
plt.imshow(clouds, cmap="jet", interpolation='nearest')

In [ ]:
false_positives = mask_clouds_persistent
clouds = mask_clouds.isel(time=i)

In [ ]:
clouds_tofill = np.where(false_positives, 0, clouds + 1)

In [ ]:
plt.imshow(clouds_tofill)
# plt.colorbar()

In [ ]:
dilation

In [ ]:
from scipy import ndimage as nd
from skimage.morphology import watershed, square
from skimage.morphology import dilation, opening

distance = nd.distance_transform_edt(false_positives)
result = watershed(-distance, dilation(clouds_tofill, square(3)), mask=false_positives, \
                   connectivity=square(3))

In [ ]:
plt.imshow(np.where(false_positives, dilation(clouds_tofill, square(50)), 0), interpolation='nearest')

In [ ]:
plt.imsave("result.png",np.where(false_positives, dilation(clouds_tofill, square(10)), 0) * 255)

In [ ]:
plt.imsave("result.png",result)

In [ ]:
plt.imshow(distance)

In [ ]:
plt.imshow(result)

In [ ]:
clouds.plot()

In [ ]:
xr.where(false_positives, result - 1, clouds).plot()

In [ ]:
clouds

In [ ]:
plt.imshow(result)

In [ ]:
# Using these peaks, label image using watershed segmentation
# mask = np.zeros(distance.shape, dtype=bool)
# mask[tuple(coords.T)] = True

labels = watershed(-distance, markers) #, mask=valid_data)

# # Reproject MNDWI surface into same resolution as distance array
# mndwi_highres = xr_reproject(mndwi_i,
#                              geobox=distance_da.geobox)

# # For each labelled region, calculate MNDWI zonal statistics 
# mndwi_regions = regionprops(label_image=labels,

In [ ]:
plt.imshow(markers)

In [ ]:
mask_clouds_mean.plot(robust=True)

In [ ]:
good_times = landsat_ds.where(~(mask_nodata | mask_clouds_persistent)
                             ).nbart_green.mean(dim=['x', 'y']) < 2500

In [ ]:
landsat_ds = landsat_ds.sel(time=good_times)
landsat_ds

In [ ]:

# local_thresh = threshold_local(mask_clouds_mean, block_size=333, method='gaussian', offset=-0.03, param=100)
# (mask_clouds_mean > local_thresh).plot()

**Once the load is complete**, examine the data by printing it in the next cell.
The `Dimensions` argument revels the number of time steps in the data set, as well as the number of pixels in the `x` (longitude) and `y` (latitude) dimensions.

In [ ]:
# import odc.algo
    
# # Extract boolean mask
# mask = odc.algo.enum_to_bool(landsat_ds.fmask, 
#                              categories=['nodata', 'cloud', 'shadow'])

# # Close mask to remove small holes in cloud, open mask to 
# # remove narrow false positive cloud, then dilate
# mask_cleaned = odc.algo.mask_cleanup(mask, mask_filters=[('opening', 30)])

# # # Add new mask as nodata pixels
# landsat_ds = odc.algo.erase_bad(landsat_ds, mask_cleaned)
# landsat_ds = odc.algo.to_float(landsat_ds)
# landsat_ds

### Plot example timestep in true colour
To visualise the data, use the pre-loaded `rgb` utility function to plot a true colour image for a given time-step. 
White areas indicate where clouds or other invalid pixels in the image have been masked.

Change the value for `timestep` and re-run the cell to plot a different timestep (timesteps are numbered from `0` to `n_time - 1` where `n_time` is the total number of timesteps; see the `time` listing under the `Dimensions` category in the dataset print-out above).


In [ ]:
# mask.isel(time=15).plot()

In [ ]:
# # Set the timesteps to visualise
# timestep = 25

# # Generate RGB plots at each timestep
# rgb(landsat_ds, index=timestep)


## Compute Normalised Difference Water Index
To extract intertidal depth contours, we need to be able to seperate water from land in our study area. 
To do this, we can use our Landsat data to calculate a water index called the `Normalised Difference Water Index`, or NDWI. 
This index uses the ratio of green and near-infrared radiation to identify the presence of water. 
The formula is:

$$
\begin{aligned}
\text{NDWI} &= \frac{(\text{Green} - \text{NIR})}{(\text{Green} + \text{NIR})}
\end{aligned}
$$

where `Green` is the green band and `NIR` is the near-infrared band. 

When it comes to interpreting the index, **High values (greater than 0, blue colours) typically represent water pixels**, while **low values (less than 0, red colours) represent land**. 
You can use the cell below to calculate and plot one of the images after calculating the index.

In [ ]:
# Set the timesteps to visualise
timestep = 45

# Calculate the water index
landsat_ds = calculate_indices(landsat_ds,
                               index='NDWI',
                               collection='ga_s2_1')

# Plot the resulting image for the same timestep selected above
landsat_ds.NDWI.isel(time=timestep).plot(cmap='RdBu',
                                         size=6,
                                         vmin=-0.8,
                                         vmax=0.8)
plt.show()


How does the plot of the index compare to the optical image from earlier? Was there water or land anywhere you weren't expecting?

## Model tide heights
The location of the shoreline can vary greatly from low to high tide. In the code below, we aim to calculate the height of the tide at the exact moment each Landsat image was acquired. 
This will allow us to built a sorted time series of images taken at low tide to high tide, which we will use to generate the intertidal elevation model.

The `tidal_tag` function below uses the [OTPS TPXO8 tidal model](http://volkov.oce.orst.edu/tides/global.html) to calculate the height of the tide at the exact moment each satellite image in our dataset was taken, and adds this as a new `tide_height` attribute in our dataset (for more information about this function, refer to the [Tidal modelling notebook](../Frequently_used_code/Tidal_modelling.ipynb)). 

> **Note:** this function can only model tides correctly if the centre of your study area is located over water. 
If this isn't the case, you can specify a custom tide modelling location by passing a coordinate to `tidepost_lat` and `tidepost_lon` (e.g. `tidepost_lat=-27.73, tidepost_lon=153.46`).

In [ ]:
# Calculate tides for each timestep in the satellite dataset
landsat_ds = tidal_tag(ds=landsat_ds, tidepost_lat=None, tidepost_lon=None)

# Print the output dataset with new `tide_height` variable
landsat_ds


Now that we have modelled tide heights, we can plot them to visualise the range of tide that was captured by Landsat across our time series: 

In [ ]:
# Plot the resulting tide heights for each Landsat image:
landsat_ds.tide_height.plot()
plt.show()

## Create water index summary images from low to high tide

Using these tide heights, we can sort our Landsat dataset by tide height to reveal which parts of the landscape are inundated or exposed from low to high tide. 

Individual remote sensing images can be affected by noise, including clouds, sunglint and poor water quality conditions (e.g. sediment). 
To produce cleaner images that can be compared more easily between tidal stages, we can create 'summary' images or composites that combine multiple images into one image to reveal the 'typical' or median appearance of the landscape at different tidal stages. 
In this case, we use the median as the summary statistic because it prevents strong outliers (like stray clouds) from skewing the data, which would not be the case if we were to use the mean.

In the code below, we take the time series of images, sort by tide and categorise each image into 9 discrete tidal intervals, ranging from the lowest (tidal interval 1) to the highest tides observed by Landsat (tidal interval 9). 
For more information on this method, refer to [Sagar et al. 2018](https://www.sciencedirect.com/science/article/pii/S0034425717301591).

In [ ]:
# classes
n = 10

# Sort every image by tide height
landsat_ds = landsat_ds.sortby('tide_height')

# Bin tide heights into 9 tidal intervals from low (1) to high tide (9)
binInterval = np.linspace(landsat_ds.tide_height.min(),
                          landsat_ds.tide_height.max(),
                          num=n)
tide_intervals = pd.cut(landsat_ds.tide_height,
                        bins=binInterval,
                        labels=range(1, n),
                        include_lowest=True)

# Add interval to dataset
landsat_ds['tide_interval'] = xr.DataArray(tide_intervals, coords=[landsat_ds.time])
landsat_ds.sortby('time').tide_height.plot()
for i in binInterval: plt.axhline(i, c='black', alpha=0.5)
    


We can plot the boundaries between the nine tidal intervals on the same plot we generated earlier:

Now that we have a dataset where each image is classified into a discrete range of the tide, we can combine our images into a set of nine individual images that show where land and water is located from low to high tide. 
**This step can take several minutes to process.**

In [ ]:
# For each interval, compute the median water index and tide height value
landsat_intervals = (landsat_ds[['tide_interval', 'NDWI', 'tide_height']]
                     .compute()
                     .groupby('tide_interval')
                     .median(dim='time'))

# Plot the resulting set of tidal intervals
landsat_intervals.NDWI.plot(col='tide_interval', col_wrap=5, cmap='RdBu')
plt.show()


In [ ]:
item = ((landsat_intervals.NDWI < 0) * landsat_intervals.tide_interval).max(dim='tide_interval')
item.plot(cmap='RdYlBu_r')

The plot above should make it clear how the shape and structure of the coastline changes significantly from low to high tide as low-lying tidal flats are quickly inundated by increasing water levels.

## Extract depth contours from imagery
We now want to extract an accurate boundary between land and water for each of the tidal intervals above. 
The code below identifies the depth contours based on the boundary between land and water by tracing a line along pixels with a water index value of `0` (the boundary between land and water water index values). 
It returns a `geopandas.GeoDataFrame` with one depth contour for each tidal interval that is labelled with tide heights in metres relative to Mean Sea Level.


In [ ]:
# Set up attributes to assign to each waterline
attribute_df = pd.DataFrame({'tide_m': landsat_intervals.tide_height.values})

# Extract waterlines
contours_gdf = subpixel_contours(da=item,
                                 z_values=[i - 0.5 for i in range(1, n)],
                                 crs=landsat_ds.crs,
                                 affine=landsat_ds.geobox.transform,
                                 attribute_df=attribute_df,
                                 min_vertices=20,
                                 dim='tide_interval')

# Plot output shapefile over the top of the first tidal interval water index
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
landsat_intervals.NDWI.sel(tide_interval=1).plot(ax=ax, 
                                                 cmap='Greys', 
                                                 add_colorbar=False)
contours_gdf.plot(ax=ax, column='tide_m', cmap='YlOrRd', legend=True)
plt.show()

In [ ]:
# # Set up attributes to assign to each waterline
# attribute_df = pd.DataFrame({'tide_m': landsat_intervals.tide_height.values})

# # Extract waterlines
# contours_gdf = subpixel_contours(da=landsat_intervals.NDWI,
#                                  z_values=0,
#                                  crs=landsat_ds.crs,
#                                  affine=landsat_ds.geobox.transform,
#                                  attribute_df=attribute_df,
#                                  min_vertices=20,
#                                  dim='tide_interval')

# # Plot output shapefile over the top of the first tidal interval water index
# fig, ax = plt.subplots(1, 1, figsize=(15, 10))
# landsat_intervals.NDWI.sel(tide_interval=1).plot(ax=ax, 
#                                                  cmap='Greys', 
#                                                  add_colorbar=False)
# contours_gdf.plot(ax=ax, column='tide_m', cmap='YlOrRd', legend=True)
# plt.show()


The above plot is a basic visualisation of the depth contours returned by the `subpixel_contours` function. 
**Deeper contours (in m relative to Mean Sea Level) are coloured in yellow; more shallow contours are coloured in red.**
Now have the shapefile, we can use a more complex function to make an interactive plot for viewing the topography of the intertidal zone.

### Plot interactive map of depth contours coloured by time
The next cell provides an interactive map with an overlay of the depth contours identified in the previous cell. 
Run it to view the map.

Zoom in to the map below to explore the resulting set of depth contours. 
Using this data, we can easily identify areas of the coastline which are only exposed in the lowest of tides, or other areas that are only covered by water during high tides.

In [ ]:
contours_gdf.explore(column='tide_m')

## Interpolate contours into a Digital Elevation Model (DEM)

While the contours above provide valuable information about the topography of the intertidal zone, we can extract additional information about the 3D structure of the coastline by converting them into an elevation raster (i.e. a Digital Elevation Model or DEM).

In the cell below, we convert the shapefile above into an array of points with X, Y and Z coordinates, where the Z coordinate is the point's elevation relative to Mean Sea Level. 
We then use these XYZ points to interpolate smooth, continuous elevations across the intertidal zone using linear interpolation.

In [ ]:
# First convert our contours shapefile into an array of XYZ points
xyz_array = contours_to_arrays(contours_gdf, 'tide_m')

# Interpolate these XYZ points over the spatial extent of the Landsat dataset
intertidal_dem = interpolate_2d(ds=landsat_intervals,
                                x_coords=xyz_array[:, 0],
                                y_coords=xyz_array[:, 1],
                                z_coords=xyz_array[:, 2])

# Plot the output
intertidal_dem.plot(cmap='viridis', size=8)
plt.show()


You can see in the output above that our interpolation includes areas of ocean and land that are outside of the area affected by tides.
To clean up the data, we can restrict the DEM to only the area between the lowest and highest observed tides:

In [ ]:
# Identify areas that are always wet (e.g. below low tide), or always dry
above_lowest = landsat_intervals.isel(tide_interval=0).NDWI < 0
below_highest = landsat_intervals.isel(tide_interval=-1).NDWI > 0

# Keep only pixels between high and low tide
intertidal_dem_clean = intertidal_dem.where(above_lowest & below_highest)

# Plot the cleaned dataset
intertidal_dem_clean.plot(cmap='viridis', size=8)
plt.show()


In [ ]:
(dc.load(product='nidem', 
        like=landsat_ds.geobox).nidem-intertidal_dem_clean).plot(robust=True, vmin=-0.5, vmax=0.5, cmap='RdBu', size=10)

## Export intertidal DEM as a GeoTIFF
As a final step, we can take the intertidal DEM we created and export it as a GeoTIFF that can be loaded in GIS software like QGIS or ArcMap (to download the dataset from the DEA Sandbox, locate it in the file browser to the left, right click on the file, and select "Download").

In [ ]:
# Export as a GeoTIFF
write_cog(geo_im=intertidal_dem_clean, 
          fname='intertidal_dem.tif', 
          overwrite=True)


In [ ]:
contours_gdf.to_crs('EPSG:4326').to_file('intertidal_contours.geojson')

## Next steps
When you are done, return to the "Set up analysis" cell, modify some values (e.g. `time_range`, `lat_range`, `lon_range`) and rerun the analysis. 

If you're going to change the location, you'll need to make sure Landsat 5, 7 and 8 data is available for the new location, which you can check at the [DEA Explorer](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3/1990) (use the drop-down menu to view all Landsat products). 

### National Intertidal Digital Elevation Model
For more information about the science behind this notebook, please refer to the scientific article outlining the application of this approach to the entire Australian coastline: [Bishop-Taylor et al. 2019 Between the tides: Modelling the elevation of Australia's exposed intertidal zone at continental scale](https://doi.org/10.1016/j.ecss.2019.03.006).

***
## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2021

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)

In [ ]:
def convolve_average(array, size_m=500, resolution=10):
    
    from scipy import signal
    
    # Set window size
    window = int(size_m / resolution)
    pad_window = (window + 1)//2
    
    # Pad with edge values
    padded = np.pad(array, pad_width=pad_window, mode='edge')
    
    # Convolve
    smoothed = signal.fftconvolve(padded, 
                                  np.ones((window, window)), 
                                  mode='same') / (window * window)
    
    # Remove padding
    return smoothed[pad_window:-pad_window, pad_window:-pad_window]
    
    
    

In [ ]:
from datacube.utils.geometry import GeoBox, Geometry
from shapely.geometry import box

maxy, miny = -15.7582, -15.94627
minx, maxx = 136.63593, 136.9977

maxy, miny = (-17.5, -17.6)
minx, maxx = (139.70, 139.86)


time_range = ('2016', '2021')

In [ ]:
geom = Geometry(geom=box(minx, miny, maxx, maxy),
                crs='EPSG:4326').to_crs('EPSG:3577')

In [ ]:
# Create the 'query' dictionary object
query = {
    'geopolygon': geom.buffer(1000),
    'time': time_range,
    'measurements': ['nbart_coastal_aerosol', 'nbart_blue', 'nbart_green', 'nbart_red', 'nbart_green', 'nbart_nir_1'],
    'resolution': (-10, 10),
}

# Identify the most common projection system in the input query 
output_crs = mostcommon_crs(dc=dc, product='s2a_ard_granule', query=query)

ds = load_ard(dc=dc, 
                      products=['s2a_ard_granule', 
                                's2b_ard_granule'], 
                      output_crs=output_crs,
                      mask_pixel_quality=False,
                      group_by='solar_day',
                      min_gooddata=0.0,
                      resampling={'*': 'average', 'fmask': 'nearest'},
                      dask_chunks={'time': 1, 'x': 2048, 'y': 2048},
                      **query)

# ds.load()


In [ ]:
ds = ds.where(ds > 0)

In [ ]:
ds['nbart_coastal_aerosol'].load()

In [ ]:
band_ave = ds['nbart_coastal_aerosol'].groupby('time').apply(convolve_average)

In [ ]:
# band_ave_median = band_ave.rolling(time=75, center=True, min_periods=1).median()
band_ave_median = band_ave.median(dim=['time'])

In [ ]:
band_ave_std = band_ave.std(dim=['time'])

In [ ]:
threshold = (band_ave_std / band_ave_median)

In [ ]:
band_ave_diff = band_ave / band_ave_median

In [ ]:
(band_ave_diff.isel(time=9) > threshold).plot.imshow()

In [ ]:
# band_ave_diff = band_ave / band_ave_median

In [ ]:
# band_ave_diff.isel(time=9).plot()

In [ ]:
to_keep = (band_ave_diff.std(dim=['x','y']) > 0.5) & (band_ave_diff.mean(dim=['x','y']) > 2)

In [ ]:
ds['cloud_mask'] = band_ave_diff

In [ ]:
ds_noclouds = ds.sel(time=~to_keep)

In [ ]:
landsat_ds = ds.where(ds_noclouds['cloud_mask'] < (1 + (threshold * 0.25)))

In [ ]:
landsat_ds.nbart_red.isel(time=45).plot()

In [ ]:
landsat_ds

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(20, 5))
time = 45
ds_noclouds[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().isel(time=time).plot.imshow(ax=axes[0], robust=True, vmin=0, vmax=2000)
# (ds_noclouds['cloud_mask'] > 1.5).isel(time=time).plot.imshow(ax=axes[1], vmin=0, vmax=1)
# (ds_noclouds['cloud_mask'] < 0.6).isel(time=time).plot.imshow(ax=axes[2], vmin=0, vmax=1)


# (np.abs(ds_noclouds['cloud_mask']) > (band_ave_std * 0.25)).isel(time=time).plot.imshow(ax=axes[1], vmin=0, vmax=1)
# (ds_noclouds['cloud_mask'] > (band_ave_std * 0.25)).isel(time=time).plot.imshow(ax=axes[2], vmin=0, vmax=1)

(ds_noclouds['cloud_mask'] > (1 + (threshold * 0.25))).isel(time=time).plot.imshow(ax=axes[1], vmin=0, vmax=1)
(ds_noclouds['cloud_mask'] < (1 - (threshold * 0.125))).isel(time=time).plot.imshow(ax=axes[2], vmin=0, vmax=1)


In [ ]:
(1 - (threshold * 0.125)).plot()

In [ ]:
(1 + (threshold * 0.25)).plot()

In [ ]:
(ds_noclouds['cloud_mask'] > (1 + (threshold * 0.25))).mean(dim='time').plot()

In [ ]:
(ds_noclouds['cloud_mask'] < (1 - (threshold * 0.125))).mean(dim='time').plot()

In [ ]:
(np.abs(ds_noclouds['cloud_mask']) > 500).mean(dim='time').plot()

In [ ]:
(band_ave_std * 0.25).plot()

In [ ]:
(ds_large['cloud_mask'] > 1.5).mean(dim='time').plot()

In [ ]:
# ds[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().isel(time=slice(0, 24)).plot.imshow(col='time', col_wrap=4, robust=True)

In [ ]:
cloud_mask = (np.abs(test_diff) < 600)

In [ ]:
testing = cloud_mask.astype(float).interp_like(ds_large.isel(time=0))

In [ ]:
(testing.isel(time=0) > 0.5).plot.imshow()

In [ ]:
ds.where(np.abs(test_diff) < 600)[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().isel(time=slice(0, 24)).plot.imshow(col='time', col_wrap=4, robust=True)

In [ ]:
ds.where(np.abs(test_diff) < 750)[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().isel(time=slice(0, 24)).plot.imshow(col='time', col_wrap=4, robust=True)

In [ ]:
ds.where(np.abs(test_diff) < 500)[['nbart_red', 'nbart_green', 'nbart_blue']].to_array().isel(time=slice(0, 24)).plot.imshow(col='time', col_wrap=4, robust=True)